In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
%matplotlib notebook
import importlib
import numpy as np
import pandas as pd
import keras
import sys
from keras.layers import TimeDistributed, Dense, Dropout, GRU, Bidirectional
from keras.models import Sequential
from tracker import extractor as ext, utils, metrics, visuals

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:0C:00.0)


In [2]:
%%time
trainpath   = "data/sets/UNIF-10N-25T-25000E-235R.gz"
testpath    = "data/sets/RAMP-10N-25T-3600E-235R.gz"
train_frame = pd.read_csv(trainpath)
test_frame  = pd.read_csv(testpath)
train       = utils.list_of_groups(train_frame, group="event_id")
test        = utils.list_of_groups(test_frame,  group="event_id")
print("Train is list of {0} events ({1} bytes).".format(len(train), sys.getsizeof(train)))
print("Test is list of {0} events ({1} bytes).".format(len(test), sys.getsizeof(test)))
if (not utils.is_prepared(train_frame)) or (not utils.is_prepared(test_frame)):
    print("Warning: frame is not prepared.")
    print("Look at the prepare_frame() function in tracker/extractor.py")
else:
    del train_frame
    del test_frame # To save on memory space.

Train is list of 25000 events (200320 bytes).
Test is list of 3600 events (30120 bytes).
CPU times: user 10.5 s, sys: 312 ms, total: 10.8 s
Wall time: 10.8 s


In [8]:
train = train[:100]
test  = test[:100]
order = ["phi", "r", "z"]
input_shape  = (235, 3)
n_categories = 25 + 2
optimizer    = keras.optimizers.RMSprop(lr=0.001)
histories    = []

In [101]:
def arbitrary_function(string):
    arbitrary_function.message += string
    return 0

arbitrary_function.message = ""
arbitrary_function("This message")
print(arbitrary_function.message)
arbitrary_function(" is now complete.")
print(arbitrary_function.message)

This message
This message is now complete.


In [94]:
import keras.backend as K

class Loss:
    def __init__(self):
        self.number_of_times_called = 0
        
    def __call__(self, y_true, y_pred):
        self.number_of_times_called += 1
        return K.sum(K.log(y_true) - K.log(y_pred))

custom_loss = Loss()
custom_loss.__name__ = "custom_loss"

In [95]:
model = Sequential()
model = Sequential()
model.add(Dropout(rate=1/32, input_shape=input_shape))
model.add(TimeDistributed(Dense(units=n_categories, kernel_initializer="uniform", activation="softmax")))

In [96]:
model.compile(loss=custom_loss, optimizer=optimizer, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_17 (Dropout)         (None, 235, 3)            0         
_________________________________________________________________
time_distributed_17 (TimeDis (None, 235, 27)           108       
Total params: 108
Trainable params: 108
Non-trainable params: 0
_________________________________________________________________


In [97]:
%%time
epochs     = 3
batch_size = 10
histories.append(model.fit_generator(
    ext.input_output_generator(train, batch_size, order),
    steps_per_epoch=len(train) // batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=ext.input_output_generator(test, batch_size, order),
    validation_steps=len(test) // batch_size,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=10, verbose=1),
        keras.callbacks.ReduceLROnPlateau(patience=5, verbose=1)]))

Epoch 1/3
10/10 [==============================] - 1s - loss: nan - acc: 0.0320 - val_loss: nan - val_acc: 0.0361
Epoch 2/3
10/10 [==============================] - 1s - loss: nan - acc: 0.0352 - val_loss: nan - val_acc: 0.0360
Epoch 3/3
10/10 [==============================] - 1s - loss: nan - acc: 0.0352 - val_loss: nan - val_acc: 0.0360
CPU times: user 4.34 s, sys: 300 ms, total: 4.64 s
Wall time: 4.35 s


In [98]:
print(custom_loss.number_of_times_called)

1
